This file take the blurbs scraped from a major book website and cleans them. Process is as follows:
1. Cleaning the URLs of the blurbs that were scraped so we can compare it with the 'title' column in our book-crossings dataset
2. Using a function that crawls through the URLs and matches them to titles, shifting the cells up and down throughout the process. This is because the scrape skipped many titles and added nonsense data randomly.
3. Many of the blurbs have repeated sections that I identified and deleted.
4. For each blurb, break into sentences, generate BERT vectors using BERT-as-a-service
5. Add vectors into the books dataframe and save as a CSV file

In [1]:
import numpy as np
import pandas as pd
import string
import re
import time
import random

from bert_serving.client import BertClient

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

/anaconda3/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [13]:
### Data

# BookCrossing
# http://www2.informatik.uni-freiburg.de/~cziegler/BX/

df_ratings = pd.read_csv('/DataScience/BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding = "latin-1")
df_users = pd.read_csv('/DataScience/BX-CSV-Dump/BX-Users.csv', sep=';', encoding='latin-1')
df_books = pd.read_csv('/DataScience/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding = "latin-1")

# Renaming columns for ease of use, and dropping image links I wont be using
df_ratings.rename(columns={'User-ID': 'User', 'Book-Rating': 'Rating'}, inplace=True)
df_users.rename(columns={'User-ID': 'User'}, inplace=True)
df_books.rename(columns={'ISBN': 'ISBN', 'Book-Title': 'Title', 'Year-Of-Publication': 'Year', 'Book-Author': 'Author'}, inplace=True)
df_books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], 1, inplace=True)

# Using only explicit ratings. 0 ratings are 'implicit' in documentation of data:
df_ratings = df_ratings[df_ratings['Rating'] > 0]

# Inner join df_ratings and df_books
df_ratings = df_ratings[df_ratings['ISBN'].isin(df_books.ISBN.unique())]
df_books = df_books[df_books['ISBN'].isin(df_ratings.ISBN.unique())]

df_books.reset_index(inplace=True, drop=True)
df_ratings.reset_index(inplace=True, drop=True)
df_users.reset_index(inplace=True, drop=True)

# 2 batches of scraped blurbs from Goodreads
first_scrape = pd.read_csv('/Users/jdobrow/Code/blurbs1.csv')
second_scrape = pd.read_csv('/Users/jdobrow/Code/blurbs2.csv')


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


In [165]:
# We want to add our scraped blurbs to df_books, but the scraped data is messy and
# does not line up correctly.

# Our titles from the scraped URLs. They almost, but not quite match the actual titles.

scrapes = [first_scrape, second_scrape]
url_titles = []
for chunk in scrapes:
    for book in range(len(chunk)):
        skip, url_split = 0, -1
        for i in range(len(chunk.URL[book])):
            skip += 1 if chunk.URL[book][i] == '.':
                if skip == 3:
                    url_split = chunk.URL[book][i+1:].lower()
                    url_split = re.sub(r'[^\w\s]',' ',url_split).split('_')
        if url_split == -1:
            for i in range(len(chunk.URL[book])):
                if chunk.URL[book][i] == '-':
                    url_split = chunk.URL[book][i+1:].lower().split('-')
                    break
        url_titles.append(url_split)

    # The actual text of the blurb
    blurbs = list(chunk.text)

    # The titles from our data, tidied up a bit to assist matching
    if len(chunk) == len(second_scrape):
        book_titles = list(df_books['Title'][100000:])
        for i in range(len(df_books) - len(chunk) - 100000):
            url_titles.append(None)
            blurbs.append(None)
    else:
        book_titles = list(df_books['Title'][:100000])
        for i in range(len(100000 - len(chunk)):
            url_titles.append(None)
            blurbs.append(None)
                       
    for title in range(len(book_titles)):
        book_titles[title] = book_titles[title].lower()
        book_titles[title] = re.sub(r'[^\w\s]',' ', book_titles[title]).split()

    
compare_df = pd.DataFrame()
compare_df['URLTitle'] = url_titles
compare_df['BookTitle'] = book_titles
compare_df['Blurb'] = blurbs

In [166]:
# Checks if the title and url match each other. Matches are determined by how many
# shared words there are
stop_words = ['the', 'of', 'if', 'and', 'it', 'as', 'or']
def check_matched_up(url_index, book_index, data):
    try:
        count = 0
        for k in data.URLTitle[url_index]:
            if k in data.BookTitle[book_index]:
                if k not in stop_words:
                    count += 1
        if count >= max(min(len(data.URLTitle[url_index])//2, len(data.BookTitle[book_index])//2), 1):
            return True
        else:
            return False
   
    except:
        return False  

In [213]:
# Iterates through the data and check if titles match up, and if not corrects them by
# shifting all of the data up or down or in some cases swapping entries.
goto = len(compare_df)
beginning = 0
while beginning < goto:

    for row in range(beginning, goto):
        if (check_matched_up(row, row, compare_df) == True):
            # everything good
            #print(row, 'case 1')
            None
        elif (check_matched_up(row, row + 1, compare_df) == True) and (check_matched_up(row + 1, row, compare_df) == True):
            # 2 adjacent rows need to be swapped
            url1, url2 = compare_df.URLTitle[row], compare_df.URLTitle[row + 1]
            blurb1, blurb2 = compare_df.Blurb[row], compare_df.Blurb[row + 1]
            compare_df.iloc[row, 0], compare_df.iloc[row + 1, 0] = url2, url1
            compare_df.iloc[row, 2], compare_df.iloc[row + 1, 2] = blurb2, blurb1
            #print(row, 'case 2')
            break
        elif (check_matched_up(row + 1, row, compare_df) == True):
            # Insert an empty entry into book titles to match things up
            book_titles.insert(row, None)
            url_titles.append(None)
            blurbs.append(None)
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
            #print(row, 'case 3')
            break
        elif (check_matched_up(row, row + 1, compare_df) == True):
            # Insert an empty entry into url titles to match things up
            url_titles.insert(row, None)
            blurbs.insert(row, None)
            book_titles.append(None)
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
            #print(row, 'case 4')
            break
        else:
            book_titles[row] = None
            url_titles[row] = None
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
            #print(row, 'case 5')
    
    beginning = row + 1

88160 case 5
88161 case 5
88162 case 4
88164 case 5
88165 case 5
88167 case 5
88168 case 4
88170 case 5
88171 case 5
88172 case 5
88173 case 5
88174 case 5
88175 case 5
88176 case 5
88177 case 5
88178 case 5
88179 case 5
88180 case 5
88181 case 5
88182 case 5
88183 case 5
88184 case 5
88185 case 5
88186 case 5
88187 case 5
88188 case 5
88189 case 5
88190 case 5
88191 case 5
88192 case 5
88193 case 5
88194 case 5
88195 case 5
88196 case 5
88197 case 5
88198 case 5
88199 case 5
88200 case 5
88201 case 5
88202 case 5
88203 case 5
88204 case 5
88205 case 5
88206 case 5
88207 case 5
88208 case 5
88209 case 5
88210 case 5
88211 case 5
88212 case 5
88213 case 5
88214 case 5
88215 case 5
88216 case 5
88217 case 5
88218 case 5
88219 case 5
88220 case 5
88221 case 5
88222 case 5
88223 case 4
88225 case 5
88226 case 5
88227 case 5
88228 case 5
88229 case 5
88230 case 5
88231 case 5
88232 case 5
88233 case 5
88234 case 5
88235 case 5
88236 case 5
88237 case 5
88238 case 5
88239 case 5
88240 case 5

93352 case 5
93353 case 5
93354 case 5
93355 case 5
93356 case 5
93357 case 5
93358 case 5
93359 case 5
93360 case 5
93361 case 5
93362 case 5
93363 case 5
93364 case 5
93365 case 5
93366 case 5
93367 case 5
93368 case 5
93369 case 5
93370 case 5
93371 case 5
93372 case 5
93373 case 5
93374 case 5
93375 case 5
93376 case 5
93377 case 5
93378 case 5
93379 case 5
93380 case 5
93381 case 5
93382 case 5
93383 case 5
93384 case 5
93385 case 5
93386 case 5
93387 case 5
93388 case 5
93389 case 5
93390 case 5
93391 case 5
93392 case 5
93393 case 5
93394 case 5
93395 case 5
93396 case 5
93397 case 5
93398 case 5
93399 case 5
93400 case 5
93401 case 5
93402 case 5
93403 case 5
93404 case 5
93405 case 5
93406 case 5
93407 case 5
93408 case 5
93409 case 5
93410 case 5
93411 case 5
93412 case 5
93413 case 5
93414 case 5
93415 case 5
93416 case 5
93417 case 5
93418 case 5
93419 case 5
93420 case 5
93421 case 5
93422 case 5
93423 case 5
93424 case 5
93425 case 5
93426 case 5
93427 case 5
93428 case 5

93995 case 5
93996 case 5
93997 case 5
93998 case 5
93999 case 5
94000 case 5
94001 case 5
94002 case 5
94003 case 5
94004 case 5
94005 case 5
94006 case 5
94007 case 5
94008 case 5
94009 case 5
94010 case 5
94011 case 5
94012 case 5
94013 case 5
94014 case 5
94015 case 5
94016 case 5
94017 case 5
94018 case 5
94019 case 5
94020 case 5
94021 case 5
94022 case 5
94023 case 5
94024 case 5
94025 case 5
94026 case 5
94027 case 5
94028 case 5
94029 case 5
94030 case 5
94031 case 5
94032 case 5
94033 case 5
94034 case 5
94035 case 4
94037 case 5
94038 case 5
94039 case 5
94040 case 5
94041 case 5
94042 case 5
94043 case 5
94044 case 5
94045 case 5
94046 case 5
94047 case 5
94048 case 5
94049 case 5
94050 case 5
94051 case 5
94052 case 5
94053 case 5
94054 case 5
94055 case 5
94056 case 3
94058 case 5
94059 case 5
94060 case 5
94061 case 5
94062 case 5
94063 case 5
94064 case 5
94065 case 5
94066 case 5
94067 case 5
94068 case 5
94069 case 5
94070 case 5
94071 case 5
94072 case 5
94073 case 5

94646 case 5
94647 case 5
94648 case 5
94649 case 5
94650 case 5
94651 case 5
94652 case 5
94653 case 5
94654 case 5
94655 case 5
94656 case 5
94657 case 5
94658 case 5
94659 case 5
94660 case 5
94661 case 5
94662 case 5
94663 case 5
94664 case 5
94665 case 5
94666 case 5
94667 case 5
94668 case 5
94669 case 5
94670 case 5
94671 case 5
94672 case 5
94673 case 5
94674 case 5
94675 case 5
94676 case 5
94677 case 5
94678 case 5
94679 case 5
94680 case 5
94681 case 5
94682 case 5
94683 case 5
94684 case 5
94685 case 5
94686 case 5
94687 case 5
94688 case 5
94689 case 5
94690 case 5
94691 case 5
94692 case 5
94693 case 5
94694 case 5
94695 case 5
94696 case 5
94697 case 5
94698 case 5
94699 case 5
94700 case 5
94701 case 5
94702 case 5
94703 case 5
94704 case 5
94705 case 5
94706 case 5
94707 case 5
94708 case 5
94709 case 5
94710 case 5
94711 case 5
94712 case 5
94713 case 5
94714 case 5
94715 case 5
94716 case 5
94717 case 5
94718 case 5
94719 case 5
94720 case 3
94722 case 5
94723 case 5

95290 case 5
95291 case 5
95292 case 5
95293 case 5
95294 case 5
95295 case 4
95297 case 5
95298 case 5
95299 case 5
95300 case 5
95301 case 5
95302 case 5
95303 case 5
95304 case 5
95305 case 5
95306 case 5
95307 case 5
95308 case 5
95309 case 5
95310 case 5
95311 case 5
95312 case 5
95313 case 5
95314 case 5
95315 case 5
95317 case 5
95318 case 5
95319 case 5
95320 case 5
95321 case 5
95322 case 5
95324 case 5
95325 case 5
95326 case 5
95327 case 5
95328 case 5
95329 case 5
95330 case 5
95331 case 5
95332 case 5
95333 case 5
95334 case 5
95335 case 5
95336 case 5
95337 case 5
95338 case 5
95339 case 5
95340 case 5
95341 case 5
95342 case 5
95343 case 5
95344 case 5
95345 case 5
95346 case 5
95347 case 5
95348 case 5
95349 case 5
95350 case 5
95351 case 5
95352 case 5
95353 case 5
95354 case 5
95355 case 5
95356 case 5
95357 case 5
95358 case 5
95359 case 5
95360 case 5
95361 case 5
95362 case 5
95363 case 5
95364 case 5
95365 case 5
95366 case 5
95367 case 5
95368 case 5
95369 case 5

...

173756 case 5
173757 case 5
173758 case 5
173759 case 5
173760 case 5
173761 case 5
173762 case 5
173763 case 5
173764 case 5
173765 case 5
173766 case 5
173767 case 5
173768 case 5
173769 case 5
173770 case 5
173771 case 5
173772 case 5
173773 case 5
173774 case 5
173775 case 5
173776 case 5
173777 case 5
173778 case 5
173779 case 5
173780 case 5
173781 case 5
173782 case 5
173783 case 5
173784 case 5
173785 case 5
173786 case 5
173787 case 5
173788 case 5
173789 case 5
173790 case 5
173791 case 5
173792 case 5
173793 case 5
173794 case 5
173795 case 5
173796 case 5
173797 case 5
173798 case 5
173799 case 5
173800 case 5
173801 case 5
173802 case 5
173803 case 5
173804 case 5
173805 case 5
173806 case 5
173807 case 5
173808 case 5
173809 case 5
173810 case 5
173811 case 5
173812 case 5
173813 case 5
173814 case 5
173815 case 5
173816 case 5
173817 case 5
173818 case 5
173819 case 5
173820 case 5
173821 case 5
173822 case 5
173823 case 5
173824 case 5
173825 case 5
173826 case 5
173827

174343 case 5
174344 case 5
174345 case 5
174346 case 5
174347 case 5
174348 case 5
174349 case 5
174350 case 5
174351 case 5
174352 case 5
174353 case 5
174354 case 5
174355 case 5
174356 case 5
174357 case 5
174358 case 5
174359 case 5
174360 case 5
174361 case 5
174362 case 5
174363 case 5
174364 case 5
174365 case 5
174366 case 5
174367 case 5
174368 case 5
174369 case 5
174370 case 5
174371 case 5
174372 case 5
174373 case 5
174374 case 5
174375 case 5
174376 case 5
174377 case 5
174378 case 5
174379 case 5
174380 case 5
174381 case 5
174382 case 5
174383 case 5
174384 case 5
174385 case 5
174386 case 5
174387 case 5
174388 case 5
174389 case 5
174390 case 5
174391 case 5
174392 case 5
174393 case 5
174394 case 5
174395 case 5
174396 case 5
174397 case 5
174398 case 5
174399 case 5
174400 case 5
174401 case 5
174402 case 5
174403 case 5
174404 case 5
174405 case 5
174406 case 5
174407 case 5
174408 case 5
174409 case 5
174410 case 5
174411 case 5
174412 case 5
174413 case 5
174414

174929 case 5
174930 case 5
174931 case 5
174932 case 5
174933 case 5
174934 case 5
174935 case 5
174936 case 5
174937 case 5
174938 case 5
174939 case 5
174940 case 5
174941 case 5
174942 case 5
174943 case 5
174944 case 5
174945 case 5
174946 case 5
174947 case 5
174948 case 5
174949 case 5
174950 case 5
174951 case 5
174952 case 5
174953 case 5
174954 case 5
174955 case 5
174956 case 5
174957 case 5
174958 case 5
174959 case 5
174960 case 5
174961 case 5
174962 case 5
174963 case 5
174964 case 5
174965 case 5
174966 case 5
174967 case 5
174968 case 5
174969 case 5
174970 case 5
174971 case 5
174972 case 5
174973 case 5
174974 case 5
174975 case 5
174976 case 5
174977 case 5
174978 case 5
174979 case 5
174980 case 5
174981 case 5
174982 case 5
174983 case 5
174984 case 5
174985 case 5
174986 case 5
174987 case 5
174988 case 5
174989 case 5
174990 case 5
174991 case 5
174992 case 5
174993 case 5
174994 case 5
174995 case 5
174996 case 5
174997 case 5
174998 case 5
174999 case 5
175000

175515 case 5
175516 case 5
175517 case 5
175518 case 5
175519 case 5
175520 case 5
175521 case 5
175522 case 5
175523 case 5
175524 case 5
175525 case 5
175526 case 5
175527 case 5
175528 case 5
175529 case 5
175530 case 5
175531 case 5
175532 case 5
175533 case 5
175534 case 5
175535 case 5
175536 case 5
175537 case 5
175538 case 5
175539 case 5
175540 case 5
175541 case 5
175542 case 5
175543 case 5
175544 case 5
175545 case 5
175546 case 5
175547 case 5
175548 case 5
175549 case 5
175550 case 5
175551 case 5
175552 case 5
175553 case 5
175554 case 5
175555 case 5
175556 case 5
175557 case 5
175558 case 5
175559 case 5
175560 case 5
175561 case 5
175562 case 5
175563 case 5
175564 case 5
175565 case 5
175566 case 5
175567 case 5
175568 case 5
175569 case 5
175570 case 5
175571 case 5
175572 case 5
175573 case 5
175574 case 5
175575 case 5
175576 case 5
175577 case 5
175578 case 5
175579 case 5
175580 case 5
175581 case 5
175582 case 5
175583 case 5
175584 case 5
175585 case 5
175586

176101 case 5
176102 case 5
176103 case 5
176104 case 5
176105 case 5
176106 case 5
176107 case 5
176108 case 5
176109 case 5
176110 case 5
176111 case 5
176112 case 5
176113 case 5
176114 case 5
176115 case 5
176116 case 5
176117 case 5
176118 case 5
176119 case 5
176120 case 5
176121 case 5
176122 case 5
176123 case 5
176124 case 5
176125 case 5
176126 case 5
176127 case 5
176128 case 5
176129 case 5
176130 case 5
176131 case 5
176132 case 5
176133 case 5
176134 case 5
176135 case 5
176136 case 5
176137 case 5
176138 case 5
176139 case 5
176140 case 5
176141 case 5
176142 case 5
176143 case 5
176144 case 5
176145 case 5
176146 case 5
176147 case 5
176148 case 5
176149 case 5
176150 case 5
176151 case 5
176152 case 5
176153 case 5
176154 case 5
176155 case 5
176156 case 5
176157 case 5
176158 case 5
176159 case 5
176160 case 5
176161 case 5
176162 case 5
176163 case 5
176164 case 5
176165 case 5
176166 case 5
176167 case 5
176168 case 5
176169 case 5
176170 case 5
176171 case 5
176172

In [215]:
# Drop missing rows
compare_df = compare_df.dropna()
compare_df.reset_index(inplace=True, drop=True)

In [247]:
# Move blurbs from compare_df to a new dataframe to work out of

active = df_books.dropna().reset_index(drop=True)
blurbs = []
blurb_row = 0
for book_row in range(len(active)):
    cleaned_title = active['Title'][book_row].lower()
    cleaned_title = re.sub(r'[^\w\s]',' ', cleaned_title).split()
    if cleaned_title == compare_df.BookTitle[blurb_row]:
        blurbs.append(compare_df.Blurb[blurb_row])
        if blurb_row < len(compare_df):
            blurb_row += 1
    else:
        blurbs.append(None)
active['Blurb'] = blurbs

active.dropna(inplace=True)
active.reset_index(inplace=True, drop=True)


In [251]:
# Due to the nature of the scraped data, many of the blurbs have a repeated section.
# This block finds the repeat and deletes it.

updated_blurbs = []

for row in range(len(active)):
    # Most of the very short blurbs appear to be nonsense
    if len(active.Blurb[row]) > 100:
        try:
            # A lot of random case by case errors are happening so I catch all with a
            # try, and when it fails just go with the raw blurb since it's close enough.
            go = False
            string = active.Blurb[row][200:]
            last_i = -1
            first_i = 0
            # the regular expression search doesn't like non alphanumeric characters so this
            # searches for a chunk to find a repeating section.
            while (go == False) and (first_i < 190):
                first_char = '!'
                first_i = last_i + 1
                while not first_char.isalpha():
                    first_char = active.Blurb[row][first_i]
                    first_i += 1
                last_char = 'a'
                last_i = first_i + 1
                str_len = 0
                while (last_char.isalpha() or last_char.isspace()) and (str_len < 10):
                    str_len = last_i - first_i + 1
                    last_char = active.Blurb[row][last_i]
                    last_i += 1
                if str_len == 10:
                    go = True
                if last_i > len(active.Blurb[row]) - 2:
                    go = True
            a = re.search(r'{}'.format(active.Blurb[row][(first_i-1):(last_i-1)]), string)
            
            updated_blurbs.append(active.Blurb[row][a.start() + 200 - first_i + 1:])
            
        except:
            updated_blurbs.append(active.Blurb[row])
    else:
        updated_blurbs.append(None)
        
active['Blurb'] = updated_blurbs
active = active.dropna()
active.reset_index(inplace=True, drop=True)


In [258]:
# Get the BERT vectors for each sentence in a blurb, and then average them

bc = BertClient()

vector_list = []
for blurb in active.Blurb:
    raw = re.split('\. |! |\? ', blurb)
    sentences = []
    for sentence in raw:
        if (len(sentence) > 0) and not sentence.isspace():
            sentences.append(sentence)
    vectors = bc.encode(sentences)
    
    mean_vector = vectors[0]
    for i in range(len(vectors) - 1):
        mean_vector = mean_vector + vectors[i + 1]
    mean_vector = mean_vector/len(vectors)
    vector_list.append(mean_vector)

/anaconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


1
501
1001
1501
2001
2501
3001
3501
4001
4501
5001
5501
6001
6501
7001
7501
8001
8501
9001
9501
10001
10501
11001
11501
12001
12501
13001
13501
14001
14501
15001
15501
16001
16501
17001
17501
18001
18501
19001
19501
20001
20501
21001
21501


In [270]:
# Combine books with vectors and save to a local csv file

vectors = pd.DataFrame(vector_list)
active = pd.concat([active, vectors], axis=1)
active.to_csv('/DataScience/Final Capstone Files/books_with_blurbs_and_BERT_combined.csv')
